### RDD缓存  保留血缘关系

分散存储在服务器内部

In [ ]:
rdd.cache() #内存缓存

In [ ]:
rdd.persist(StorageLevel.MEMORY_ONLY)  #只内存缓存

In [ ]:
rdd.persist(StorageLevel.DISK_ONLY)  #只硬盘缓存

In [ ]:
rdd.persist(StorageLevel.MEMORY_AND_DISK)  #内存不够,硬盘缓存

In [ ]:
rdd.persist(StorageLevel.OFF_HEAP)  #内存不够,硬盘缓存

In [ ]:
rdd.unpersist() #主动清理缓存

## 演示

In [1]:
from pyspark import SparkConf,SparkContext

In [2]:
from pyspark.storagelevel import StorageLevel

In [3]:
conf=SparkConf().setAppName('spark4').setMaster('local[*]')

In [4]:
sc=SparkContext(conf=conf)

22/05/11 13:18:56 WARN Utils: Your hostname, MaQiang resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
22/05/11 13:18:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/11 13:18:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
rdd=sc.textFile('txt/spark3.txt')

In [6]:
rdd1=rdd.flatMap(lambda x:x.split(" "))

In [7]:
rdd2=rdd1.map(lambda x:(x,1))

In [8]:
rdd2.persist(StorageLevel.MEMORY_AND_DISK)
# rdd2.cache()

PythonRDD[2] at RDD at PythonRDD.scala:53

In [9]:
rdd3=rdd2.reduceByKey(lambda x,y:x+y)

In [10]:
rdd3.collect()

[('test', 3),
 ('qiang', 2),
 ('hadoop', 1),
 ('hello', 3),
 ('txt', 2),
 ('spark', 2)]

In [11]:
rdd4=rdd2.groupByKey()

In [12]:
rdd4.collect()

[('test', <pyspark.resultiterable.ResultIterable at 0x7f02c0a99280>),
 ('qiang', <pyspark.resultiterable.ResultIterable at 0x7f02c0a99550>),
 ('hadoop', <pyspark.resultiterable.ResultIterable at 0x7f02c0a995b0>),
 ('hello', <pyspark.resultiterable.ResultIterable at 0x7f02c0a99610>),
 ('txt', <pyspark.resultiterable.ResultIterable at 0x7f02c0a99670>),
 ('spark', <pyspark.resultiterable.ResultIterable at 0x7f02c0a996d0>)]

In [13]:
rdd2.unpersist()

PythonRDD[2] at RDD at PythonRDD.scala:53

## 不保留血缘关系备份 : CheckPoint

集中储存在hdfs

In [16]:
rdd1=rdd.flatMap(lambda x:x.split(" "))

In [17]:
rdd2=rdd1.map(lambda x:(x,1))

In [20]:
sc.setCheckpointDir('checkpoint')

In [21]:
rdd2.checkpoint()

In [25]:
rdd3=rdd2.reduceByKey(lambda x,y:x+y) #直接从checkpoint 开始

In [26]:
rdd3.collect()

[('test', 3),
 ('qiang', 2),
 ('hadoop', 1),
 ('hello', 3),
 ('txt', 2),
 ('spark', 2)]